# Autoencoder_Ensemble_Net

## Importing necessary libraries and setting environ variables

In [214]:
import os
import sys
import time
import math
import random
import numpy as np
import matplotlib.pylab as plt

# Try importing tqdm notebook
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError as e:
    print('tdqm.notebook not found. Try updating tdqm. Reverting to base tqdm',e)
    import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

# CUSTOM IMPORTS
import LoaderFish                                               # function to gen custom point clouds (Wang et. al)
from utils import chamfer_loss                                  # utility functions

# TODO UNCOMMENT when implemented
# from utils import chamfer_loss, HyperParameter, DirectorySetting

# TODO UNCOMMENT when implemented
# from nn_modles import AutoDecoder, CompNet, EnsembleCompNet       # autodecoder, comp_net, and ensemble_compnet modules
# from dataset_module import PointDriftDS, EncodingDS, EncodingDS   # dataset modules


%matplotlib inline

tdqm.notebook not found. Try updating tdqm. Reverting to base tqdm No module named 'tqdm.notebook'


## GPU Device Configuration

Then, we set up and configure our computational devices: Whether we use GPU or perform the calculation on CPU. we use the torch.devices() and torch.cude.is_available() functions to configure our computational devices

**Note: GPU Training is required**

In [215]:
device = None
use_cuda = torch.cuda.is_available()

if use_cuda:
    # inbuilt cudnn auto-tuner searches for best algorithm for hardware
    # cuddn.benchmark should be set to True when our input size does not vary
    torch.backends.cudnn.benchmark = True
    print("GPU training available")
    device = torch.device("cuda:0")
    print(f"Index of CUDA device in use is {torch.cuda.current_device()}")
else:
    print("GPU training NOT available")
    device = torch.device("cpu")
    print("Can only train on CPU")

GPU training available
Index of CUDA device in use is 0


## Hyper parameter Class

# TODO save in a separate Python file

Important hyperparameters

-   latent Encoding dimension
-   Encoding iterations and learning rate
-   No of points sampled from point cloud
-   No of sample shapes used in each class
-   L2 regularization for encodings

In [216]:
class HyperParameter:
    def __init__(self,
                 l2_reg=None,
                 encoding_size=256,
                 encoding_iters=50,
                 num_point_cloud=3,
                 epochs=4,
                 lr=0.00001,
                 batch_size=32):

        self.l2_reg = l2_reg
        self.learning_rate = lr
        self.encoding_size = encoding_size
        self.encoding_iters = encoding_iters
        self.num_point_cloud = num_point_cloud
        self.epochs = epochs
        self.batch_size = batch_size

    def __repr__(self):
        return f"l2_reg: {self.l2_reg}\n" + \
               f"learning_rate: {self.learning_rate}\n" + \
               f"encoding_size: {self.encoding_size}\n" + \
               f"encoding_iters: {self.encoding_iters}\n" + \
               f"num_point_cloud: {self.num_point_cloud}\n" + \
               f"epochs: {self.epochs}\n" + \
               f"batch_size: {self.batch_size}\n"


class DirectorySetting:

    def __init__(self,
                 DATA_DIR="./data",
                 OUTPUT_DIR="./tranformed/",
                 AUTODECODER_TRAINED_WEIGHT_DIR="./autodecoder_trained_weights",
                 CLASSIFIER_TRAINED_WEIGHT_DIR="./classifier_trained_weights"):

        self.AUTODECODER_TRAINED_WEIGHT_DIR = AUTODECODER_TRAINED_WEIGHT_DIR
        self.CLASSIFIER_TRAINED_WEIGHT_DIR = CLASSIFIER_TRAINED_WEIGHT_DIR
        self.OUTPUT_DIR = OUTPUT_DIR
        self.DATA_DIR = DATA_DIR

        os.makedirs(self.AUTODECODER_TRAINED_WEIGHT_DIR, exist_ok=True)
        os.makedirs(self.CLASSIFIER_TRAINED_WEIGHT_DIR, exist_ok=True)
        os.makedirs(self.OUTPUT_DIR, exist_ok=True)

    def __repr__(self):
        return f"DATA_DIR: {self.DATA_DIR}\n" + \
               f"OUTPUT_DIR: {self.OUTPUT_DIR}\n" + \
               f"AUTODECODER_TRAINED_WEIGHT_DIR: {self.AUTODECODER_TRAINED_WEIGHT_DIR}\n" + \
               f"CLASSIFIER_TRAINED_WEIGHT_DIR: {self.CLASSIFIER_TRAINED_WEIGHT_DIR}\n"

## Set HyperParameter, DirectorySetting, SEED, and DEBUG Status

In [217]:
DEBUG = True
SEED  = 17*19

HP = HyperParameter(epochs=10,
                    l2_reg=None, 
                    batch_size=16,
                    num_point_cloud=3,
                    encoding_iters=1000, 
                    encoding_size=256)
DS = DirectorySetting()

np.random.seed(SEED)
torch.manual_seed(SEED)
if use_cuda:
    torch.cuda.manual_seed(SEED)

## Training steps for autodecoder_ensemble_net

1)   Train AutoDecoder

2)   Find dataset latent encodings (latent representations of a point drift between two objects) and store

3)   Train an ensemble of CompNets for 3D object point cloud classification

## Neural Network Models

# TODO save in a separate Python file

In [277]:
class AutoDecoder(nn.Module):
    """
    AutoDecoder NN to learn point drift (latent encoding) between two 3D shapes
    """

    def __init__(self,  encoding_dim=256, point_dim=3):
        super(AutoDecoder, self).__init__()
        self.fc1 = nn.Conv1d(encoding_dim + point_dim, 128, 1)
        self.fc2 = nn.Conv1d(128, 64, 1)
        self.fc3 = nn.Conv1d(64, point_dim, 1)

    def forward(self, X, encoding):
        num_points = X.shape[-1]  # num of points in each shape
        enc = encoding.unsqueeze(-1).repeat(1, 1, num_points)
        X_enc = torch.cat([X, enc], 1)
        X_enc = F.leaky_relu(self.fc1(X_enc))
        X_enc = F.leaky_relu(self.fc2(X_enc))

        # Return the drift from obj X determined by the latent encoding
        return X + self.fc3(X_enc)


class CompNet(nn.Module):
    """
    Ingests the latent encoding of two 3D objects 
    and outputs the similarity score
    """

    def __init__(self, encoding_size=256):
        super(CompNet, self).__init__()
        self.fc1 = nn.Linear(encoding_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, encoding):
        X = F.leaky_relu(self.fc1(encoding))
        return torch.sigmoid(self.fc2(X))


class EnsembleCompNet(nn.Module):
    """
    Ingests the latent encoding of two 3D objects 
    and outputs the similarity score using an ensemble of CompNets
    Stacked Ensemble
    """

    def __init__(self, comp_net=CompNet, num_ensemble=5, encoding_dim=256, seed_val=SEED):
        """
        if comp_net is a module, EnsembleCompNet creates num_ensemble*comp_net NN modules
        if comp_net is a list of modules, EnsembleCompNet iterates through comp_net to get the NN modules
        """
        super(EnsembleCompNet, self).__init__()
        self.ensemble_compnet = nn.ModuleList()

        if isinstance(comp_net, list):
            if num_ensemble != len(comp_net):
                raise IndexError(
                    f"Length of comp_nets: {len(comp_net)} and num_ensemble: {num_ensemble} do not match")
            comp_net_list = comp_net
            for i in range(num_ensemble):
                self.ensemble_compnet.append(comp_net_list[i])
        else:
            for i in range(num_ensemble):
                torch.manual_seed(seed_val*i+1)
                if use_cuda:
                    torch.cuda.manual_seed(seed_val*i+1)
                self.ensemble_compnet.append(comp_net(encoding_dim))
        self.final = nn.Linear(num_ensemble, 1)

    def forward(self, encoding):
        """ Returns the final value of the results after a nn.Linear layer """
        total_pred = torch.cat([net(encoding)
                                for net in self.ensemble_compnet])
        total_pred = total_pred.reshape(-1, len(self.ensemble_compnet))

        return torch.sigmoid(self.final(total_pred))

In [218]:
""" DEBUG Section """
if DEBUG:
    def test_nn_modules():
        autodecoder = AutoDecoder()
        compnet = CompNet(256)
        ensemblecompnet1 = EnsembleCompNet()
        ensemblecompnet2 = EnsembleCompNet([CompNet(256), CompNet(256)], num_ensemble=2)
        compnet.cuda()
        autodecoder.cuda()
        ensemblecompnet1.cuda()
        ensemblecompnet2.cuda()
        print(compnet(torch.Tensor(np.random.randn(1, 256)).cuda()))
        print(ensemblecompnet2(torch.Tensor(np.random.randn(1, 256)).cuda()))
        
    test_nn_modules()

tensor([[0.5570]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.2648]], device='cuda:0', grad_fn=<SigmoidBackward>)


## Dataset Modules and DataLoader implementations

# TODO Save in a separate Python file

In [233]:
class PointNetDS(Dataset):
    """
    Create train, test dataset
    """

    def __init__(self, data, sampling_interval=3):
        # sample every sampling_interval-th point to speed up
        self.data = data.transpose((0, 2, 1))[:, :, ::sampling_interval]
        self.labels = labels.squeeze()
        self.data = torch.from_numpy(self.data).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx]


class PointDriftDS(Dataset):
    """
    Pairs each shape with one shape from the same class and one shape from a different class
    """

    def __init__(self, data, labels, sampling_interval=3):
        # sample every sampling_interval-th point to speed up
        self.data = data.transpose((0, 2, 1))[:, :, ::sampling_interval]
        self.labels = labels.squeeze()

        self.same_cls = []
        self.diff_cls = []
        idx_arr = np.arange(self.data.shape[0])
        same_idx = []
        diff_idx = []
        for i in range(self.labels.max() + 1):
            same_idx.append(idx_arr[self.labels == i])
            diff_idx.append(idx_arr[self.labels != i])
        for i in range(data.shape[0]):
            same = same_idx[self.labels[i]]
            diff = diff_idx[self.labels[i]]
            self.same_cls.append(same[random.randint(0, len(same) - 1)])
            self.diff_cls.append(diff[random.randint(0, len(diff) - 1)])
        self.data = torch.from_numpy(self.data).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        X = self.data[idx]
        same_cls_data = self.data[self.same_cls[idx]]
        diff_cls_data = self.data[self.diff_cls[idx]]

        return X, same_cls_data, diff_cls_data, idx


class EncodingDS(Dataset):
    """
    Generate encoding for each pair of shapes in the PointDriftDS
    """

    def __init__(self, PDDS, autodecoder, latent_size=256):
        self.PointDriftDS = PDDS
        self.autodecoder = autodecoder
        self.latent_size = latent_size
        self.same_cls = torch.zeros((len(self.PointDriftDS), latent_size))
        self.diff_cls = torch.zeros((len(self.PointDriftDS), latent_size))

    def train_encodings(self, num_iterations=50, lr=0.01, l2_reg=False, batch_size=16):
        dl = DataLoader(self.PointDriftDS,
                        batch_size=batch_size, shuffle=False)
        i = 0
        batch_cnt = 0
        same_cls_loss = 0.0
        diff_cls_loss = 0.0
        self.autodecoder.eval()

        for batch_idx, (x, y, z, idx) in enumerate(dl):
            j = i + len(idx)
            loss, encoding = find_encoding(x, y, self.autodecoder, encoding_iters=num_iterations,
                                           encoding_size=self.latent_size, lr=lr, l2_reg=l2_reg,)
            same_cls_loss += loss
            self.same_cls[i:j] = encoding
            loss, encoding = find_encoding(x, z, self.autodecoder, encoding_iters=num_iterations,
                                           encoding_size=self.latent_size, lr=lr, l2_reg=l2_reg,)
            diff_cls_loss += loss
            self.diff_cls[i:j] = encoding

            i = j
            batch_cnt += 1
        print("Encodings trained")
        return (self.same_cls, self.diff_cls, same_cls_loss / batch_cnt, diff_cls_loss / batch_cnt)

    def __len__(self):
        return len(self.PointDriftDS)

    def __getitem__(self, idx):
        return (*self.PointDriftDS[idx], self.same_cls[idx], self.diff_cls[idx])

## Function to find encodings by using the autodecoder

In [8]:
def find_encoding(X, y, autodecoder, encoding_iters=300,
                  encoding_size=256, lr=5e-4, l2_reg=False):
    """
    Generate the encoding (latent vector) for each data in X
    """

    def _adjust_lr(initial_lr, optimizer, num_iters, decreased_by, adjust_lr_every):

        lr = initial_lr * ((1 / decreased_by) **
                           (num_iters // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr

    decreased_by = 10
    adjust_lr_every = encoding_iters // 2

    encoding = torch.ones(X.shape[0], encoding_size).normal_(
        mean=0, std=1.0 / math.sqrt(encoding_size)).cuda()

    encoding.requires_grad = True
    optimizer = torch.optim.Adam([encoding], lr=lr)
    loss_num = 0

    for i in range(encoding_iters):
        autodecoder.eval()
        _adjust_lr(lr, optimizer, i, decreased_by, adjust_lr_every)
        optimizer.zero_grad()
        y_pred = autodecoder(X, encoding)
        loss = chamfer_loss(y_pred, y, ps=y.shape[-1])

        if l2_reg:
            loss += 1e-4 * torch.mean(encoding.pow(2))
        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(i, loss.cpu().data.numpy(), encoding.norm())
        loss_num = loss.cpu().data.numpy()

    return loss_num, encoding

In [9]:
def train_decoder(HP, DS, train_ds, test_ds=None, decoder=None, save_weight=True):
    """ 
    Default training is for 3D point dimensions
    
    Suggested Settings
        EPOCHS = 10
        point_dim = 3
        batch_size = 16
        learning_rate = 0.001
        encoding_size = 256
    """
    EPOCHS = HP.epochs
    point_dim = HP.num_point_cloud
    batch_size = HP.batch_size
    encoding_size = HP.encoding_size
    lr = HP.learning_rate
    
    if decoder is None:
        adnet = AutoDecoder(encoding_size, point_dim)
    else:
        adnet = decoder
    adnet = adnet.cuda()

    # encodings for same class transformation
    same_encoding = torch.nn.Embedding(
        len(train_ds), encoding_size, max_norm=1.0)
    # init encoding with Kaiming Initialization
    torch.nn.init.normal_(same_encoding.weight.data,
                          0.0,
                          1.0 / math.sqrt(encoding_size))

    # encodings for different class transformation
    diff_encoding = torch.nn.Embedding(
        len(train_ds), encoding_size, max_norm=1.0)
    # init encoding with Kaiming Initialization
    torch.nn.init.normal_(diff_encoding.weight.data,
                          0.0,
                          1.0 / math.sqrt(encoding_size))

    optimizer = torch.optim.Adam([
        {"params": adnet.parameters(), "lr": lr, },
        {"params": same_encoding.parameters(), "lr": lr, },
        {"params": diff_encoding.parameters(), "lr": lr, }, ])

    op_schedule = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    adnet = nn.DataParallel(adnet)
    adnet.cuda()

    data_loader_train = DataLoader(train_ds, batch_size=batch_size,
                                   shuffle=True)
    
    for epoch in range(0, EPOCHS):
        adnet.train()
        same_total_loss = 0.0
        diff_total_loss = 0.0

        for batch_idx, (x, y, z, idx) in enumerate(data_loader_train):
            optimizer.zero_grad()
            x, y, z = x.cuda(), y.cuda(), z.cuda()
            x, y, z = (Variable(x).float(),
                       Variable(y).float(),
                       Variable(z).float())
            y_pred = adnet(x, same_encoding(torch.LongTensor(idx)))
            loss_cham = chamfer_loss(y, y_pred, ps=y.shape[-1])
            same_total_loss += loss_cham.data.cpu().numpy()
            loss_cham.backward()

            z_pred = adnet(x, diff_encoding(torch.LongTensor(idx)))
            loss_cham = chamfer_loss(z, z_pred, ps=z.shape[-1])
            diff_total_loss += loss_cham.data.cpu().numpy()
            loss_cham.backward()

            optimizer.step()
            if batch_idx % 100 == 0 and batch_idx > 0:
                print(f"Epoch: {epoch}. batch_idx: {batch_idx}")
                print("Loss: ",same_total_loss / 100, diff_total_loss / 100)
                same_total_loss = 0.0
                diff_total_loss = 0.0
        op_schedule.step(epoch)

        if test_ds is not None and epoch % 5 == 0:
            print("Eval: ", eval_decoder(adnet, test_ds, batch_size=batch_size))

    if save_weight:
        torch.save(adnet.state_dict(), DS.AUTODECODER_TRAINED_WEIGHT_DIR + '/decoder.pth')
    return adnet

In [10]:
def eval_decoder(decoder, eval_ds, batch_size=16):
    decoder.eval()
    encoding_ds = EncodingDataset(eval_ds, decoder)
    return encoding_ds.train_encodings(num_iterations=10, lr=0.05, batch_size=batch_size)[2:]

In [261]:
def train_compnet(HP, DS, train_ds, test_ds=None, compnet=None, save_weights=True):
    """
    Train the CompNet

    Suggested Parameters
    EPOCHS=10
    batch_size=16
    encoding_size=256
    learning_rate=0.001
    """
    EPOCHS = HP.epochs
    point_dim = HP.num_point_cloud
    batch_size = HP.batch_size
    encoding_size = HP.encoding_size
    lr = HP.learning_rate

    if compnet is None:
        cpnet = CompNet(encoding_size=encoding_size)
    else:
        cpnet = compnet
    cpnet = cpnet.cuda()

    optimizer = torch.optim.Adam(cpnet.parameters(), lr=lr)
    op_schedule = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    cpnet = nn.DataParallel(cpnet)
    cpnet.cuda()

    data_loader_train = DataLoader(train_ds, batch_size=batch_size,
                    shuffle=True)

    loss_fn = nn.BCELoss()
    for epoch in range(EPOCHS):
        same_total_loss = 0.0
        diff_total_loss = 0.0
        cpnet.train()
        for batch_idx, (x, y, z, idx, same_cls, diff_cls) in enumerate(data_loader_train):
            optimizer.zero_grad()

            same_cls, diff_cls = same_cls.cuda(), diff_cls.cuda()
            same_cls, diff_cls = Variable(
                same_cls).float(), Variable(diff_cls).float()

            same_pred = cpnet(same_cls)
            same_target = torch.ones(same_pred.shape).float().cuda()
            same_loss = loss_fn(same_pred, same_target)
            same_loss.backward()
            same_total_loss += same_loss.data.cpu().numpy()

            diff_pred = cpnet(diff_cls)
            diff_target = torch.zeros(diff_pred.shape).float().cuda()
            diff_loss = loss_fn(diff_pred, diff_target)
            diff_loss.backward()
            diff_total_loss += diff_loss.data.cpu().numpy()

            optimizer.step()
            if batch_idx % 100 == 0 and batch_idx > 0:
                print(f"Epoch: {epoch}. batch_idx: {batch_idx}")
                print("Loss: ", same_total_loss / 100, diff_total_loss / 100)
                same_total_loss = 0.0
                diff_total_loss = 0.0
        op_schedule.step(epoch)
        
        if test_ds is not None and epoch % 5 == 0:
            print("Eval: ", eval_compnet(cpnet, test_ds, batch_size=batch_size))

    if save_weights:
        torch.save(cpnet.state_dict(),
                   DS.CLASSIFIER_TRAINED_WEIGHT_DIR + '/compnet.pth')
    return cpnet

In [162]:
def eval_compnet(cpnet, test_ds, batch_size=16, pred_threshold=0.5):
    cpnet.eval()
    test_dl = DataLoader(test_ds, batch_size=batch_size,
                         shuffle=False)
    loss_fn = nn.BCELoss()

    same_total_loss = 0.0
    diff_total_loss = 0.0
    batch_cnt = 0
    same_corr_cnt = 0.0
    diff_corr_cnt = 0.0
    same_incorr_cnt = 0.0
    diff_incorr_cnt = 0.0

    for batch_idx, (x, y, z, idx, same_cls, diff_cls) in enumerate(test_dl):
        batch_cnt += 1
        same_cls, diff_cls = same_cls.cuda(), diff_cls.cuda()

        same_pred = cpnet(same_cls)
        same_target = torch.ones(same_pred.shape).float().cuda()
        same_loss = loss_fn(same_pred, same_target)
        same_total_loss += same_loss.data.cpu().numpy()

        same_corr_cnt += np.sum(same_pred.detach().cpu().numpy()
                                > pred_threshold)
        same_incorr_cnt += np.sum(same_pred.detach().cpu().numpy()
                                  <= pred_threshold)

        diff_pred = cpnet(diff_cls)
        diff_target = torch.zeros(diff_pred.shape).float().cuda()
        diff_loss = loss_fn(diff_pred, diff_target)
        diff_total_loss += diff_loss.data.cpu().numpy()

        diff_corr_cnt += np.sum(diff_pred.detach().cpu().numpy()
                                < pred_threshold)
        diff_incorr_cnt += np.sum(diff_pred.detach().cpu().numpy()
                                  >= pred_threshold)

    precision = same_corr_cnt / (same_corr_cnt+diff_incorr_cnt)
    recall = same_corr_cnt / (same_corr_cnt+same_incorr_cnt) # same_corr_cnt / len(test_ds)
    print("------------------ Evaluation Report ------------------")
    print(f"Total Accuracy: {(same_corr_cnt+diff_corr_cnt)/(2*len(test_ds))}")
    print(f"After {batch_cnt} batches and {len(test_ds)} test points")
    print()
    
    print(f"Metrics for the same class:")
    print(f"Avg loss: {same_total_loss / batch_cnt}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {(2*precision*recall)/(precision+recall)}")

    precision = diff_corr_cnt / (diff_corr_cnt+same_incorr_cnt) 
    recall = diff_corr_cnt / (diff_corr_cnt+diff_incorr_cnt) # diff_corr_cnt / len(test_ds)
    print()
    print(f"Metrics for the diff class:")
    print(f"Avg loss: {diff_total_loss / batch_cnt}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {(2*precision*recall)/(precision+recall)}")

    return (same_total_loss, diff_total_loss,
            same_corr_cnt, diff_corr_cnt,
            same_incorr_cnt, diff_incorr_cnt,
            batch_cnt, len(test_ds))

In [255]:
def classify(X, cls_samples, cls_num, autodecoder, compnet,
             num_iterations=15, latent_size=256, lr=0.01, l2_reg=False):
    autodecoder.eval()
    compnet.eval()
    num_samples = len(cls_samples) # 6
    num_X = len(X)
    print("Orig X Shape:", X.shape) # 16 3 683
    X = X.unsqueeze(1).repeat((1, num_samples, 1, 1))
    # print(X.shape) # [16, 6, 3, 683]
    # print(X.reshape((X.shape[0]*num_samples, 1, *X.shape[2:])).shape) # [96, 1, 3, 683]
    X = X.reshape((X.shape[0]*num_samples, 1, *X.shape[2:])).squeeze()
    print("X shape", X.shape, "num_X:", num_X) # [96, 3, 683], 16
    cls_samples = cls_samples.repeat(num_X, 1, 1)
    print("cls_samples Shape", cls_samples.shape) # [96, 3, 683]

    loss, encoding = find_encoding(X, cls_samples, autodecoder, encoding_iters=num_iterations,
                                   encoding_size=latent_size, lr=lr, l2_reg=l2_reg)
    print(loss, 'ecnoding=;',encoding.shape) # 96 x 256
    preds = compnet(encoding)
    print(preds.shape) # 96,1
    preds = preds.reshape(
        (num_X, cls_num, (preds.shape[0]//(num_X * cls_num))))  # 16, 3, 96/(16x3)
    
    print("preds shape", preds.shape) # 16 3 2
    return preds

In [256]:
gds = PointNetDS(pn_test_data)
dl1 = DataLoader(pn_test_ds, batch_size=16, shuffle=False)
X = next(iter(dl1))[0]

X = gds[(pn_test_labels == 2).squeeze()]
cls_samples = gds[(pn_test_labels == 2).squeeze()][:3]

classify(X, cls_samples, 1, pn_autodecoder, compnet1)

Orig X Shape: torch.Size([11, 3, 683])
X shape torch.Size([33, 3, 683]) num_X: 11
cls_samples Shape torch.Size([33, 3, 683])
0 0.050085843 tensor(5.7709, device='cuda:0', grad_fn=<NormBackward0>)
0.011123054 ecnoding=; torch.Size([33, 256])
torch.Size([33, 1])
preds shape torch.Size([11, 1, 3])


tensor([[[0.9311, 0.7207, 0.4000]],

        [[0.5229, 0.4717, 0.5597]],

        [[0.9470, 0.7850, 0.6468]],

        [[0.9074, 0.6133, 0.5556]],

        [[0.6011, 0.7495, 0.5542]],

        [[0.6198, 0.6978, 0.7069]],

        [[0.6561, 0.4018, 0.3689]],

        [[0.4924, 0.7097, 0.5966]],

        [[0.0916, 0.3650, 0.5170]],

        [[0.3604, 0.4960, 0.1065]],

        [[0.5914, 0.5729, 0.0479]]], device='cuda:0',
       grad_fn=<AsStridedBackward>)

_______

# Testing our AutoDecoder

## Generating dummy Train and Test pair Fish shapes using LoaderFish (Wang et al. 2019)

In [15]:
CD_before = []  # chamfer distance before registration
CD_after = []  # chamfer distance after registration

train_size = 20000  # size for training data
test_size = 1000    # size for testing data
deformation_list = [0.5]
deformation = deformation_list[0]

print(".......Synthesizing Training Pairs......")
lf_train = LoaderFish.PointRegDataset(total_data=train_size,
                                      deform_level=deformation,
                                      noise_ratio=0,
                                      outlier_ratio=0,
                                      outlier_s=False,
                                      outlier_t=True,
                                      noise_s=False,
                                      noise_t=True,
                                      missing_points=0,
                                      miss_source=False,
                                      miss_targ=True)

data_loader_lf_train = torch.utils.data.DataLoader(
    lf_train, batch_size=16, shuffle=True)

print(".......Synthesizing Testing Pairs......")
lf_test = LoaderFish.PointRegDataset(total_data=test_size,
                                     deform_level=deformation,
                                     noise_ratio=0,
                                     outlier_ratio=0,
                                     outlier_s=False,
                                     outlier_t=True,
                                     noise_s=False,
                                     noise_t=True,
                                     missing_points=0,
                                     miss_source=False,
                                     miss_targ=True)

data_loader_lf_test = torch.utils.data.DataLoader(
    lf_test, batch_size=10, shuffle=False)

  0%|          | 73/20000 [00:00<00:27, 726.52it/s]

.......Synthesizing Training Pairs......


 15%|█▍        | 148/1000 [00:00<00:01, 738.45it/s]

.......Synthesizing Testing Pairs......


100%|██████████| 1000/1000 [00:01<00:00, 736.47it/s]


In [16]:
sample_data = next(iter(data_loader_lf_train))
print(len(sample_data), sample_data[0].shape, len(sample_data[0]), sample_data[0][0].shape)

5 torch.Size([16, 2, 91]) 16 torch.Size([2, 91])


## Training the AutoDecoder on the sample generated LoaderFish dataset

We generate encoding latent vectors for the shapes generated by the LoaderFish

In [17]:
# Hyperparameter Setup for loaderfish test
EPOCHS = 4
lr = 0.0001
encoding_size = 256
scheduler_gamma = 0.5
scheduler_step_sz = 30

In [18]:
adnet = AutoDecoder(point_dim=2)
adnet.cuda()

encoding = torch.nn.Embedding(len(lf_train), encoding_size, max_norm=1.0)
# init encoding with Kaiming Initialization
torch.nn.init.normal_(
    encoding.weight.data,
    0.0,
    1.0 / math.sqrt(encoding_size))

optimizer = torch.optim.Adam([{"params": adnet.parameters(), "lr": lr, },
                           {"params": encoding.parameters(), "lr": lr, }, ])
op_schedule = optim.lr_scheduler.StepLR(
    optimizer, step_size=scheduler_step_sz, gamma=scheduler_gamma)

# use multiple gpus
adnet = nn.DataParallel(adnet)
adnet.cuda()

# Train the AutoDecoder adnet on the training data
for epoch in range(EPOCHS):
    adnet.train()
    total_loss = 0.0
    for batch_idx, (X, y, theta, _, idx) in enumerate(data_loader_lf_train):
        X, y, theta = X.cuda(), y.cuda(), theta.cuda()
        X, y, theta = Variable(X).float(), Variable(y).float(), Variable(theta).float()
        y_pred = adnet(X, encoding(torch.LongTensor(idx)))
        loss_cham = chamfer_loss(y, y_pred, ps=y.shape[-1])
        
        optimizer.zero_grad()
        loss_cham.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Epoch:{epoch} Batch Index:{batch_idx}")
            print(loss_cham.data.cpu().numpy())
    op_schedule.step(epoch)


Epoch:0 Batch Index:0
0.0526277
Epoch:0 Batch Index:100
0.04193649
Epoch:0 Batch Index:200
0.04914833
Epoch:0 Batch Index:300
0.04160332
Epoch:0 Batch Index:400
0.033825263
Epoch:0 Batch Index:500
0.030017994
Epoch:0 Batch Index:600
0.028596533
Epoch:0 Batch Index:700
0.033026434
Epoch:0 Batch Index:800
0.030410869
Epoch:0 Batch Index:900
0.021873951
Epoch:0 Batch Index:1000
0.021696791
Epoch:0 Batch Index:1100
0.022542069
Epoch:0 Batch Index:1200
0.019152097
Epoch:1 Batch Index:0
0.016948704
Epoch:1 Batch Index:100
0.017050937
Epoch:1 Batch Index:200
0.01621691
Epoch:1 Batch Index:300
0.015167228
Epoch:1 Batch Index:400
0.015761562
Epoch:1 Batch Index:500
0.014706012
Epoch:1 Batch Index:600
0.01617067
Epoch:1 Batch Index:700
0.015201665
Epoch:1 Batch Index:800
0.013097441
Epoch:1 Batch Index:900
0.015565841
Epoch:1 Batch Index:1000
0.013944613
Epoch:1 Batch Index:1100
0.012908214
Epoch:1 Batch Index:1200
0.012408565
Epoch:2 Batch Index:0
0.012165579
Epoch:2 Batch Index:100
0.013545833

In [19]:
# Save weights of the decoder
torch.save(adnet.state_dict(), DS.AUTODECODER_TRAINED_WEIGHT_DIR+'/loaderfish_encoder01.pth')

## Testing the AudoDecoder network on the sample LoaderFish data

In [20]:
adnet.eval()

for batch_idx, (X, y, theta, _, idx) in enumerate(data_loader_lf_test):
    X, y, theta = X.cuda(), y.cuda(), theta.cuda()
    X, y, theta = Variable(X).float(), Variable(y).float(), Variable(theta).float()
    loss_encoding, generated_encoding = find_encoding(X, y, adnet, 
                                             encoding_iters=300, 
                                             encoding_size=256)
    
    y_pred = adnet(X, generated_encoding)
    loss_cham = chamfer_loss(y, y_pred, ps=y.shape[-1])
    if batch_idx % 30 == 0:
        print(f"At batch:{batch_idx}, Chamfer Loss:{loss_cham}, Encoding Chamfer Loss:{loss_encoding}")

0 0.008613841 tensor(3.1736, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0061344756 tensor(3.1572, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0056720297 tensor(3.1910, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0053899586 tensor(3.2523, device='cuda:0', grad_fn=<NormBackward0>)
200 0.0053690346 tensor(3.2589, device='cuda:0', grad_fn=<NormBackward0>)
250 0.005349036 tensor(3.2667, device='cuda:0', grad_fn=<NormBackward0>)
At batch:0, Chamfer Loss:0.005328803323209286, Encoding Chamfer Loss:0.005329195410013199
0 0.0072438284 tensor(3.1175, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0056943465 tensor(3.1572, device='cuda:0', grad_fn=<NormBackward0>)
100 0.005288393 tensor(3.2384, device='cuda:0', grad_fn=<NormBackward0>)
150 0.005076689 tensor(3.3354, device='cuda:0', grad_fn=<NormBackward0>)
200 0.005058905 tensor(3.3454, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0050417306 tensor(3.3560, device='cuda:0', grad_fn=<NormBackward0>)
0 0.007972601 tensor(3.1994, devi

200 0.0057919915 tensor(3.2736, device='cuda:0', grad_fn=<NormBackward0>)
250 0.005773581 tensor(3.2827, device='cuda:0', grad_fn=<NormBackward0>)
0 0.008929576 tensor(3.1081, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0075312797 tensor(3.1072, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0070575126 tensor(3.1825, device='cuda:0', grad_fn=<NormBackward0>)
150 0.006728595 tensor(3.2858, device='cuda:0', grad_fn=<NormBackward0>)
200 0.0067082355 tensor(3.2953, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0066883382 tensor(3.3057, device='cuda:0', grad_fn=<NormBackward0>)
0 0.008518103 tensor(3.1289, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0067029214 tensor(3.1194, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0062985616 tensor(3.1710, device='cuda:0', grad_fn=<NormBackward0>)
150 0.006103735 tensor(3.2522, device='cuda:0', grad_fn=<NormBackward0>)
200 0.006086816 tensor(3.2610, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0060698045 tensor(3.2709, device='cuda:0', g

100 0.0059073158 tensor(3.2662, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0056890775 tensor(3.3479, device='cuda:0', grad_fn=<NormBackward0>)
200 0.0056701973 tensor(3.3564, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0056521315 tensor(3.3655, device='cuda:0', grad_fn=<NormBackward0>)
0 0.009324845 tensor(3.1899, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0064241285 tensor(3.1910, device='cuda:0', grad_fn=<NormBackward0>)
100 0.005946964 tensor(3.2334, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0057177767 tensor(3.2831, device='cuda:0', grad_fn=<NormBackward0>)
200 0.005699134 tensor(3.2892, device='cuda:0', grad_fn=<NormBackward0>)
250 0.005679658 tensor(3.2957, device='cuda:0', grad_fn=<NormBackward0>)
0 0.007810995 tensor(3.1521, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0061094016 tensor(3.1472, device='cuda:0', grad_fn=<NormBackward0>)
100 0.005630332 tensor(3.2187, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0054397034 tensor(3.2906, device='cuda:0', g

100 0.005597581 tensor(3.1983, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0053457166 tensor(3.2695, device='cuda:0', grad_fn=<NormBackward0>)
200 0.0053282785 tensor(3.2758, device='cuda:0', grad_fn=<NormBackward0>)
250 0.005310865 tensor(3.2828, device='cuda:0', grad_fn=<NormBackward0>)
0 0.008830138 tensor(3.1318, device='cuda:0', grad_fn=<NormBackward0>)
50 0.007108544 tensor(3.1143, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0066543957 tensor(3.1423, device='cuda:0', grad_fn=<NormBackward0>)
150 0.006435905 tensor(3.2082, device='cuda:0', grad_fn=<NormBackward0>)
200 0.006418854 tensor(3.2155, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0064024064 tensor(3.2230, device='cuda:0', grad_fn=<NormBackward0>)
0 0.009050413 tensor(3.1069, device='cuda:0', grad_fn=<NormBackward0>)
50 0.005810829 tensor(3.0998, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0053099804 tensor(3.1636, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0051232246 tensor(3.2365, device='cuda:0', gra

0 0.008920657 tensor(3.1621, device='cuda:0', grad_fn=<NormBackward0>)
50 0.00696095 tensor(3.2188, device='cuda:0', grad_fn=<NormBackward0>)
100 0.006456427 tensor(3.3078, device='cuda:0', grad_fn=<NormBackward0>)
150 0.006227441 tensor(3.3856, device='cuda:0', grad_fn=<NormBackward0>)
200 0.0062106145 tensor(3.3925, device='cuda:0', grad_fn=<NormBackward0>)
250 0.006194095 tensor(3.4004, device='cuda:0', grad_fn=<NormBackward0>)
0 0.008863133 tensor(3.1265, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0065956605 tensor(3.1237, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0060301144 tensor(3.1643, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0057180743 tensor(3.2395, device='cuda:0', grad_fn=<NormBackward0>)
200 0.0056844708 tensor(3.2480, device='cuda:0', grad_fn=<NormBackward0>)
250 0.005664442 tensor(3.2550, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0077097346 tensor(3.1478, device='cuda:0', grad_fn=<NormBackward0>)
50 0.005999512 tensor(3.1532, device='cuda:0', grad_fn

200 0.00519029 tensor(3.4297, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0051757316 tensor(3.4374, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0072665764 tensor(3.1733, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0058653406 tensor(3.1695, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0055492143 tensor(3.2230, device='cuda:0', grad_fn=<NormBackward0>)
150 0.0053625684 tensor(3.3072, device='cuda:0', grad_fn=<NormBackward0>)
200 0.005347906 tensor(3.3161, device='cuda:0', grad_fn=<NormBackward0>)
250 0.0053333254 tensor(3.3253, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0075794156 tensor(3.2020, device='cuda:0', grad_fn=<NormBackward0>)
50 0.0058167763 tensor(3.2284, device='cuda:0', grad_fn=<NormBackward0>)
100 0.0054079457 tensor(3.2822, device='cuda:0', grad_fn=<NormBackward0>)
150 0.005222482 tensor(3.3496, device='cuda:0', grad_fn=<NormBackward0>)
200 0.005208645 tensor(3.3566, device='cuda:0', grad_fn=<NormBackward0>)
250 0.005194139 tensor(3.3634, device='cuda:0', g

# Testing our Model with PointNet Data

## Load PointNet Data

In [14]:
pn_train_data   = np.load("./data/PointNetData/train/pntcloud_7.npy")
pn_train_labels = np.load("./data/PointNetData/train/label_7.npy")
pn_test_data    = np.load("./data/PointNetData/test/pntcloud_7.npy")
pn_test_labels  = np.load("./data/PointNetData/test/label_7.npy")

pn_train_data.shape, pn_train_labels.shape, pn_test_data.shape, pn_test_labels.shape

((10817, 2048, 3), (10817, 1), (2507, 2048, 3), (2507, 1))

In [22]:
pn_train_full        = np.load("./data/PointNetData/train/pntcloud_full.npy")
pn_train_full_labels = np.load("./data/PointNetData/train/label_full.npy")
pn_test_full         = np.load("./data/PointNetData/test/pntcloud_full.npy")
pn_test_full_labels  = np.load("./data/PointNetData/test/label_full.npy")

In [23]:
pn_train_full.shape, pn_train_full_labels.shape, pn_test_full.shape, pn_test_full_labels.shape

((12137, 2048, 3), (12137, 1), (2874, 2048, 3), (2874, 1))

In [126]:
pn_train_ds = PointDriftDS(pn_train_data, pn_train_labels)
pn_test_ds = PointDriftDS(pn_test_data, pn_test_labels)

## Train AutoDecoder

**Note:** Training takes a while

In [32]:
adnet_HP = HyperParameter(lr=0.001)
adnet_HP

l2_reg: None
learning_rate: 0.001
encoding_size: 256
encoding_iters: 50
num_point_cloud: 3
epochs: 4
batch_size: 32

In [33]:
pn_autodecoder = None
load_saved_weight = True

if load_saved_weight:
    try:
        pn_autodecoder = AutoDecoder(HP.encoding_size, HP.num_point_cloud)
        pn_autodecoder.load_state_dict(torch.load(DS.AUTODECODER_TRAINED_WEIGHT_DIR+'/decoder.pth'))
    except RuntimeError as e:
        print(e)
        pn_autodecoder = train_decoder(adnet_HP, DS, train_ds=pn_train_ds, test_ds=pn_test_ds)
else:
    pn_autodecoder = train_decoder(adnet_HP, DS, train_ds=pn_train_ds, test_ds=pn_test_ds)

Error(s) in loading state_dict for AutoDecoder:
	Missing key(s) in state_dict: "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias", "fc3.weight", "fc3.bias". 
	Unexpected key(s) in state_dict: "module.fc1.weight", "module.fc1.bias", "module.fc2.weight", "module.fc2.bias", "module.fc3.weight", "module.fc3.bias". 
Epoch: 0. batch_idx: 100
Loss:  0.02694659827277064 0.07462658740580082
Epoch: 0. batch_idx: 200
Loss:  0.027464010566473008 0.06767778392881155
Epoch: 0. batch_idx: 300
Loss:  0.02799369568005204 0.06513361304998398
0 0.032697514 tensor(7.1102, device='cuda:0', grad_fn=<NormBackward0>)
0 0.060688097 tensor(7.1478, device='cuda:0', grad_fn=<NormBackward0>)
0 0.021846473 tensor(7.1961, device='cuda:0', grad_fn=<NormBackward0>)
0 0.064103164 tensor(7.1548, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03029972 tensor(7.1403, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05798519 tensor(6.9900, device='cuda:0', grad_fn=<NormBackward0>)
0 0.025432939 tensor(7.1767, device='cuda:0', 

0 0.024375262 tensor(7.2185, device='cuda:0', grad_fn=<NormBackward0>)
0 0.073100045 tensor(7.1052, device='cuda:0', grad_fn=<NormBackward0>)
0 0.027965395 tensor(7.1525, device='cuda:0', grad_fn=<NormBackward0>)
0 0.076703556 tensor(7.0231, device='cuda:0', grad_fn=<NormBackward0>)
0 0.035414733 tensor(7.1703, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05669683 tensor(7.0978, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0392856 tensor(7.0838, device='cuda:0', grad_fn=<NormBackward0>)
0 0.063177064 tensor(7.0725, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03655006 tensor(7.0806, device='cuda:0', grad_fn=<NormBackward0>)
0 0.060560666 tensor(7.1087, device='cuda:0', grad_fn=<NormBackward0>)
0 0.032653697 tensor(7.1649, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05973863 tensor(7.0466, device='cuda:0', grad_fn=<NormBackward0>)
0 0.025305294 tensor(7.1787, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055644214 tensor(7.1856, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0345678

In [253]:
""" Get memory usage of active variables """
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('pn_train_data', 265838720),
 ('pn_test_data', 61612160),
 ('idx', 20152),
 ('pn_train_labels', 10929),
 ('pn_test_labels', 2619),
 ('DataLoader', 1464),
 ('AutoDecoder', 1184),
 ('CompNet', 1184),
 ('EnsembleCompNet', 1184),
 ('HyperParameter', 1184),
 ('Dataset', 1056),
 ('DirectorySetting', 1056),
 ('EncodingDS', 1056),
 ('EncodingDataset', 1056),
 ('GenericDS', 1056),
 ('PointDriftDS', 1056),
 ('PointNetDS', 1056),
 ('Variable', 1056),
 ('diff_idx', 192),
 ('same_idx', 192),
 ('chamfer_loss', 136),
 ('classify', 136),
 ('eval_compnet', 136),
 ('eval_decoder', 136),
 ('find_encoding', 136),
 ('its_fucking_raw', 136),
 ('test_nn_modules', 136),
 ('train_compnet', 136),
 ('train_decoder', 136),
 ('d', 96),
 ('labels', 96),
 ('F', 80),
 ('cudnn', 80),
 ('nn', 80),
 ('np', 80),
 ('optim', 80),
 ('plt', 80),
 ('test_result', 80),
 ('train_result', 80),
 ('X', 72),
 ('a', 72),
 ('aa', 72),
 ('cls_samples', 72),
 ('x', 72),
 ('diff_cls', 64),
 ('same_cls', 64),
 ('DS', 56),
 ('HP', 56),


In [35]:
# Evaluate Decoder
print(eval_decoder(pn_autodecoder, pn_test_ds))

0 0.04245077 tensor(5.0333, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0559642 tensor(5.0682, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029363818 tensor(5.0582, device='cuda:0', grad_fn=<NormBackward0>)
0 0.054658405 tensor(5.0524, device='cuda:0', grad_fn=<NormBackward0>)
0 0.02842458 tensor(5.0882, device='cuda:0', grad_fn=<NormBackward0>)
0 0.057541903 tensor(5.0220, device='cuda:0', grad_fn=<NormBackward0>)
0 0.025425551 tensor(5.0640, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061649907 tensor(5.0695, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03431777 tensor(4.9510, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061498683 tensor(5.0661, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03070212 tensor(5.0931, device='cuda:0', grad_fn=<NormBackward0>)
0 0.053091783 tensor(5.0606, device='cuda:0', grad_fn=<NormBackward0>)
0 0.030673219 tensor(4.9287, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06652331 tensor(5.0520, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03247469 

0 0.05921052 tensor(5.1334, device='cuda:0', grad_fn=<NormBackward0>)
0 0.030303774 tensor(5.0341, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06480897 tensor(5.0151, device='cuda:0', grad_fn=<NormBackward0>)
0 0.027073935 tensor(5.0470, device='cuda:0', grad_fn=<NormBackward0>)
0 0.058161903 tensor(5.0387, device='cuda:0', grad_fn=<NormBackward0>)
0 0.035327677 tensor(5.0665, device='cuda:0', grad_fn=<NormBackward0>)
0 0.057656344 tensor(5.0362, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029650614 tensor(4.9892, device='cuda:0', grad_fn=<NormBackward0>)
0 0.071847275 tensor(5.0079, device='cuda:0', grad_fn=<NormBackward0>)
0 0.043887194 tensor(5.0918, device='cuda:0', grad_fn=<NormBackward0>)
0 0.053608917 tensor(5.0921, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028803578 tensor(5.0116, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04267385 tensor(5.2007, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03195124 tensor(4.9870, device='cuda:0', grad_fn=<NormBackward0>)
0 0.056514

0 0.05088862 tensor(5.0270, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05899048 tensor(5.1139, device='cuda:0', grad_fn=<NormBackward0>)
0 0.027993439 tensor(5.0518, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055730727 tensor(5.0632, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05837634 tensor(5.0183, device='cuda:0', grad_fn=<NormBackward0>)
0 0.053183794 tensor(5.0969, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03027418 tensor(4.9910, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05655142 tensor(5.0248, device='cuda:0', grad_fn=<NormBackward0>)
0 0.041968122 tensor(4.9824, device='cuda:0', grad_fn=<NormBackward0>)
0 0.056949988 tensor(5.0882, device='cuda:0', grad_fn=<NormBackward0>)
0 0.040239304 tensor(4.9939, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05088125 tensor(4.9821, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03179521 tensor(5.0045, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0698159 tensor(5.0462, device='cuda:0', grad_fn=<NormBackward0>)
0 0.051831514 t

## Encode pairs in dataset and train Encodings with trained AutoDecoders

In [36]:
train_encoding_ds = EncodingDS(PointDriftDS(pn_train_data, pn_train_labels), pn_autodecoder)
train_result = train_encoding_ds.train_encodings(num_iterations=15, lr=0.05)

0 0.032529186 tensor(5.0401, device='cuda:0', grad_fn=<NormBackward0>)
0 0.050932795 tensor(5.1038, device='cuda:0', grad_fn=<NormBackward0>)
0 0.034167647 tensor(4.9956, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07213158 tensor(4.9627, device='cuda:0', grad_fn=<NormBackward0>)
0 0.038839873 tensor(5.0723, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06123399 tensor(5.0462, device='cuda:0', grad_fn=<NormBackward0>)
0 0.045951586 tensor(5.0231, device='cuda:0', grad_fn=<NormBackward0>)
0 0.053147204 tensor(5.1215, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028430967 tensor(5.1001, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06833577 tensor(5.0784, device='cuda:0', grad_fn=<NormBackward0>)
0 0.038768623 tensor(4.8984, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06709451 tensor(4.9809, device='cuda:0', grad_fn=<NormBackward0>)
0 0.036277864 tensor(5.0551, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06888675 tensor(5.0901, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0391285

0 0.038022753 tensor(5.0863, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0742639 tensor(5.0312, device='cuda:0', grad_fn=<NormBackward0>)
0 0.022308918 tensor(5.0297, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055431984 tensor(5.0756, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03234293 tensor(5.0142, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0762884 tensor(5.0159, device='cuda:0', grad_fn=<NormBackward0>)
0 0.046983767 tensor(5.0803, device='cuda:0', grad_fn=<NormBackward0>)
0 0.044718765 tensor(5.0586, device='cuda:0', grad_fn=<NormBackward0>)
0 0.026398914 tensor(5.0212, device='cuda:0', grad_fn=<NormBackward0>)
0 0.047207598 tensor(5.0613, device='cuda:0', grad_fn=<NormBackward0>)
0 0.036190324 tensor(5.0857, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05543442 tensor(4.9396, device='cuda:0', grad_fn=<NormBackward0>)
0 0.027880248 tensor(4.9841, device='cuda:0', grad_fn=<NormBackward0>)
0 0.060168736 tensor(5.0461, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03606397

0 0.029002566 tensor(5.0584, device='cuda:0', grad_fn=<NormBackward0>)
0 0.048890233 tensor(5.1368, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028203465 tensor(5.0491, device='cuda:0', grad_fn=<NormBackward0>)
0 0.050233576 tensor(5.0009, device='cuda:0', grad_fn=<NormBackward0>)
0 0.039719246 tensor(4.9343, device='cuda:0', grad_fn=<NormBackward0>)
0 0.046281416 tensor(5.0586, device='cuda:0', grad_fn=<NormBackward0>)
0 0.035832535 tensor(5.1237, device='cuda:0', grad_fn=<NormBackward0>)
0 0.060002707 tensor(5.1007, device='cuda:0', grad_fn=<NormBackward0>)
0 0.041172363 tensor(5.0880, device='cuda:0', grad_fn=<NormBackward0>)
0 0.075259864 tensor(5.0449, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029564457 tensor(5.0037, device='cuda:0', grad_fn=<NormBackward0>)
0 0.047360476 tensor(5.0415, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0368851 tensor(5.0331, device='cuda:0', grad_fn=<NormBackward0>)
0 0.054960158 tensor(5.0800, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0342

0 0.04955804 tensor(5.0580, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05419884 tensor(5.0757, device='cuda:0', grad_fn=<NormBackward0>)
0 0.026668906 tensor(5.0435, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06104942 tensor(5.1225, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0487376 tensor(5.0916, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05303946 tensor(5.0855, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028226908 tensor(5.0813, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06311846 tensor(5.1841, device='cuda:0', grad_fn=<NormBackward0>)
0 0.032730166 tensor(5.0979, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055374753 tensor(5.0247, device='cuda:0', grad_fn=<NormBackward0>)
0 0.039350253 tensor(5.0761, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04915029 tensor(5.0781, device='cuda:0', grad_fn=<NormBackward0>)
0 0.02885806 tensor(5.1001, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05647045 tensor(5.0652, device='cuda:0', grad_fn=<NormBackward0>)
0 0.031087171 te

0 0.034144726 tensor(4.9978, device='cuda:0', grad_fn=<NormBackward0>)
0 0.08958808 tensor(4.9895, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03518563 tensor(4.9588, device='cuda:0', grad_fn=<NormBackward0>)
0 0.050989326 tensor(5.0857, device='cuda:0', grad_fn=<NormBackward0>)
0 0.039131537 tensor(4.9812, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07002605 tensor(5.0007, device='cuda:0', grad_fn=<NormBackward0>)
0 0.040175058 tensor(4.9942, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055718664 tensor(5.1392, device='cuda:0', grad_fn=<NormBackward0>)
0 0.048449274 tensor(5.0405, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05626098 tensor(4.9846, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05348089 tensor(5.1250, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06337779 tensor(5.0111, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029319031 tensor(5.0189, device='cuda:0', grad_fn=<NormBackward0>)
0 0.047759864 tensor(5.0871, device='cuda:0', grad_fn=<NormBackward0>)
0 0.02946279

...

0 0.03155414 tensor(4.9643, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06526656 tensor(5.0705, device='cuda:0', grad_fn=<NormBackward0>)
0 0.046879396 tensor(5.0347, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05809047 tensor(5.0092, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05024164 tensor(4.9660, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055480376 tensor(5.0861, device='cuda:0', grad_fn=<NormBackward0>)
0 0.020748625 tensor(5.1913, device='cuda:0', grad_fn=<NormBackward0>)
0 0.056425143 tensor(5.0816, device='cuda:0', grad_fn=<NormBackward0>)
0 0.030400245 tensor(5.0174, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05359852 tensor(5.0420, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03744618 tensor(5.0453, device='cuda:0', grad_fn=<NormBackward0>)
0 0.040392455 tensor(5.0997, device='cuda:0', grad_fn=<NormBackward0>)
0 0.032531418 tensor(5.1239, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07590711 tensor(5.0326, device='cuda:0', grad_fn=<NormBackward0>)
0 0.035813585

0 0.024037566 tensor(5.0036, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06716974 tensor(5.0622, device='cuda:0', grad_fn=<NormBackward0>)
0 0.042001005 tensor(5.0338, device='cuda:0', grad_fn=<NormBackward0>)
0 0.059344966 tensor(5.0587, device='cuda:0', grad_fn=<NormBackward0>)
0 0.049454987 tensor(5.0035, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055365935 tensor(5.0591, device='cuda:0', grad_fn=<NormBackward0>)
0 0.049914863 tensor(4.9523, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05213801 tensor(5.0908, device='cuda:0', grad_fn=<NormBackward0>)
0 0.026830433 tensor(5.0860, device='cuda:0', grad_fn=<NormBackward0>)
0 0.056310102 tensor(5.0160, device='cuda:0', grad_fn=<NormBackward0>)
0 0.042360883 tensor(5.0843, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06998708 tensor(4.9978, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03521442 tensor(5.0780, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061592996 tensor(5.0724, device='cuda:0', grad_fn=<NormBackward0>)
0 0.041431

0 0.045787062 tensor(4.9894, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05107644 tensor(5.1737, device='cuda:0', grad_fn=<NormBackward0>)
0 0.034816843 tensor(5.0325, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05134147 tensor(5.0650, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03052251 tensor(4.9653, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04907767 tensor(5.0826, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04388918 tensor(5.0834, device='cuda:0', grad_fn=<NormBackward0>)
0 0.050341953 tensor(5.0954, device='cuda:0', grad_fn=<NormBackward0>)
0 0.040746886 tensor(5.0920, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07545399 tensor(5.0343, device='cuda:0', grad_fn=<NormBackward0>)
0 0.039682336 tensor(4.9422, device='cuda:0', grad_fn=<NormBackward0>)
0 0.052981466 tensor(4.9538, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03807544 tensor(4.9763, device='cuda:0', grad_fn=<NormBackward0>)
0 0.059855774 tensor(5.0175, device='cuda:0', grad_fn=<NormBackward0>)
0 0.030449955

0 0.034665953 tensor(5.0397, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061525583 tensor(5.0636, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03864727 tensor(5.0297, device='cuda:0', grad_fn=<NormBackward0>)
0 0.060166728 tensor(5.0262, device='cuda:0', grad_fn=<NormBackward0>)
0 0.030239552 tensor(5.0779, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05854877 tensor(5.0375, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028398244 tensor(5.0273, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04714556 tensor(5.0336, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03292894 tensor(5.0257, device='cuda:0', grad_fn=<NormBackward0>)
0 0.043741666 tensor(5.0722, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028735366 tensor(4.9982, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05813053 tensor(5.0098, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029311989 tensor(5.0627, device='cuda:0', grad_fn=<NormBackward0>)
0 0.057496227 tensor(4.9898, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0302468

0 0.03337167 tensor(5.0705, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055968378 tensor(4.9985, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03697508 tensor(5.0452, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061428968 tensor(5.0975, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029275866 tensor(5.0290, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0540514 tensor(5.0142, device='cuda:0', grad_fn=<NormBackward0>)
0 0.036163244 tensor(4.9476, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055060234 tensor(5.0800, device='cuda:0', grad_fn=<NormBackward0>)
0 0.042271435 tensor(5.0684, device='cuda:0', grad_fn=<NormBackward0>)
0 0.043723926 tensor(5.0652, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03135756 tensor(5.0360, device='cuda:0', grad_fn=<NormBackward0>)
0 0.058332585 tensor(4.9500, device='cuda:0', grad_fn=<NormBackward0>)
0 0.027485851 tensor(5.1516, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06728072 tensor(5.0868, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04352163

In [37]:
test_encoding_ds = EncodingDS(PointDriftDS(pn_test_data, pn_test_labels), pn_autodecoder)
test_result = test_encoding_ds.train_encodings(num_iterations=15, lr=0.05)

0 0.026764631 tensor(4.9817, device='cuda:0', grad_fn=<NormBackward0>)
0 0.054824222 tensor(5.0565, device='cuda:0', grad_fn=<NormBackward0>)
0 0.031203177 tensor(5.0370, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07388829 tensor(5.0208, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04083364 tensor(5.1003, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061858054 tensor(5.1549, device='cuda:0', grad_fn=<NormBackward0>)
0 0.026042178 tensor(4.9962, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06847145 tensor(5.1118, device='cuda:0', grad_fn=<NormBackward0>)
0 0.032685135 tensor(5.1021, device='cuda:0', grad_fn=<NormBackward0>)
0 0.069376126 tensor(4.9552, device='cuda:0', grad_fn=<NormBackward0>)
0 0.039120417 tensor(5.1156, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0605455 tensor(5.0275, device='cuda:0', grad_fn=<NormBackward0>)
0 0.026757041 tensor(5.0323, device='cuda:0', grad_fn=<NormBackward0>)
0 0.050557528 tensor(5.1113, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0321810

0 0.033964828 tensor(5.0511, device='cuda:0', grad_fn=<NormBackward0>)
0 0.061209984 tensor(5.0189, device='cuda:0', grad_fn=<NormBackward0>)
0 0.042764235 tensor(5.0092, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07181399 tensor(5.0037, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03226217 tensor(5.0198, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07586773 tensor(5.0468, device='cuda:0', grad_fn=<NormBackward0>)
0 0.028946979 tensor(4.9685, device='cuda:0', grad_fn=<NormBackward0>)
0 0.055411723 tensor(5.0750, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04495098 tensor(4.9969, device='cuda:0', grad_fn=<NormBackward0>)
0 0.0753815 tensor(5.0247, device='cuda:0', grad_fn=<NormBackward0>)
0 0.024106447 tensor(5.0654, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07238311 tensor(4.9969, device='cuda:0', grad_fn=<NormBackward0>)
0 0.04800698 tensor(5.0628, device='cuda:0', grad_fn=<NormBackward0>)
0 0.06549365 tensor(5.1059, device='cuda:0', grad_fn=<NormBackward0>)
0 0.033348646 t

0 0.031404063 tensor(5.1480, device='cuda:0', grad_fn=<NormBackward0>)
0 0.044893067 tensor(5.0615, device='cuda:0', grad_fn=<NormBackward0>)
0 0.044061877 tensor(4.9687, device='cuda:0', grad_fn=<NormBackward0>)
0 0.07238374 tensor(5.0295, device='cuda:0', grad_fn=<NormBackward0>)
0 0.029386401 tensor(4.9783, device='cuda:0', grad_fn=<NormBackward0>)
0 0.05170419 tensor(5.0616, device='cuda:0', grad_fn=<NormBackward0>)
0 0.02811368 tensor(5.0409, device='cuda:0', grad_fn=<NormBackward0>)
0 0.056610197 tensor(5.0379, device='cuda:0', grad_fn=<NormBackward0>)
0 0.025119407 tensor(4.9436, device='cuda:0', grad_fn=<NormBackward0>)
0 0.059610862 tensor(4.9930, device='cuda:0', grad_fn=<NormBackward0>)
0 0.027422875 tensor(5.0062, device='cuda:0', grad_fn=<NormBackward0>)
0 0.093910426 tensor(5.0278, device='cuda:0', grad_fn=<NormBackward0>)
0 0.037004944 tensor(4.9948, device='cuda:0', grad_fn=<NormBackward0>)
0 0.059526548 tensor(5.0631, device='cuda:0', grad_fn=<NormBackward0>)
0 0.03999

## Train a single CompNet

In [40]:
cpnet_HP = HyperParameter(epochs=20, batch_size=16, lr=0.001)

In [42]:
compnet1 = train_compnet(cpnet_HP, DS, train_ds=train_encoding_ds, test_ds=test_encoding_ds)

Epoch: 0. batch_idx: 100
Loss:  0.47375884890556336 0.5985329768061638
Epoch: 0. batch_idx: 200
Loss:  0.3678618086874485 0.38697339296340943
Epoch: 0. batch_idx: 300
Loss:  0.384534215927124 0.35996296517550946
Epoch: 0. batch_idx: 400
Loss:  0.3650429932773113 0.3572743085026741
Epoch: 0. batch_idx: 500
Loss:  0.3835951338708401 0.3580167056620121
Epoch: 0. batch_idx: 600
Loss:  0.35218377351760866 0.34372315883636473
Eval:  (0.3502203288256742, 0.3461675195462385, array([0.84244116]), array([0.86238532]))
Epoch: 1. batch_idx: 100
Loss:  0.34379714585840704 0.3020548690855503
Epoch: 1. batch_idx: 200
Loss:  0.3338519622385502 0.32599783018231393
Epoch: 1. batch_idx: 300
Loss:  0.33520618721842765 0.31140077516436576
Epoch: 1. batch_idx: 400
Loss:  0.33837755732238295 0.3158174542337656
Epoch: 1. batch_idx: 500
Loss:  0.30661105684936046 0.29735548615455626
Epoch: 1. batch_idx: 600
Loss:  0.3344981502741575 0.30796699300408364
Epoch: 2. batch_idx: 100
Loss:  0.30618038833141326 0.2966

Epoch: 18. batch_idx: 300
Loss:  0.17963618656620384 0.18682645505294204
Epoch: 18. batch_idx: 400
Loss:  0.2007624620385468 0.1863664785400033
Epoch: 18. batch_idx: 500
Loss:  0.21408594775944947 0.1808680745214224
Epoch: 18. batch_idx: 600
Loss:  0.18257142029702664 0.20344378707930447
Epoch: 19. batch_idx: 100
Loss:  0.20267280858010053 0.19696943862363697
Epoch: 19. batch_idx: 200
Loss:  0.18093894310295583 0.18581741668283938
Epoch: 19. batch_idx: 300
Loss:  0.19211545936763286 0.18406942227855325
Epoch: 19. batch_idx: 400
Loss:  0.181600288413465 0.1796147649176419
Epoch: 19. batch_idx: 500
Loss:  0.1803071815147996 0.18550939083099366
Epoch: 19. batch_idx: 600
Loss:  0.17854316072538495 0.17236405927687884


In [163]:
eval_compnet(compnet1, test_encoding_ds)

------------------ Evaluation Report ------------------
Total Accuracy: 0.8835261268448344
After 157 batches and 2507 test points

Metrics for the same class:
Avg loss: 0.30215612478601706
Precision: 0.8832204065364687
Recall: 0.8839250099720782
F1 Score: 0.8835725677830941

Metrics for the diff class:
Avg loss: 0.31159593186275975
Precision: 0.8838323353293414
Recall: 0.8831272437175908
F1 Score: 0.8834796488427774


(47.438511591404676,
 48.92056130245328,
 2216.0,
 2214.0,
 291.0,
 293.0,
 157,
 2507)

In [127]:
dl1 = DataLoader(pn_test_ds, batch_size=16, shuffle=False)
dl2 = DataLoader(pn_test_ds, batch_size=6, shuffle=False)
x = next(iter(dl1))[0]
cls_samples = next(iter(dl2))[0]
classify(x, cls_samples, 3, pn_autodecoder, compnet1)

Orig X Shape: torch.Size([16, 3, 683])
X shape torch.Size([96, 3, 683]) num_X: 16
cls_samples Shape torch.Size([96, 3, 683])
0 0.036418088 tensor(9.8927, device='cuda:0', grad_fn=<NormBackward0>)
0.009972334 ecnoding=; torch.Size([96, 256])
torch.Size([96, 1])
preds shape torch.Size([16, 3, 2])


tensor([[[0.8061, 0.6746],
         [0.3498, 0.4677],
         [0.3448, 0.1721]],

        [[0.2337, 0.8045],
         [0.8321, 0.1947],
         [0.0540, 0.2732]],

        [[0.3428, 0.2974],
         [0.8226, 0.2165],
         [0.2875, 0.1139]],

        [[0.3850, 0.2115],
         [0.8655, 0.7313],
         [0.5704, 0.8183]],

        [[0.2319, 0.3773],
         [0.8274, 0.6851],
         [0.2484, 0.4167]],

        [[0.6508, 0.5369],
         [0.7720, 0.4706],
         [0.1952, 0.3059]],

        [[0.0557, 0.3426],
         [0.5807, 0.0748],
         [0.0276, 0.0970]],

        [[0.8802, 0.4073],
         [0.4812, 0.6655],
         [0.1131, 0.5441]],

        [[0.7369, 0.7889],
         [0.3450, 0.7472],
         [0.4409, 0.1384]],

        [[0.0769, 0.1446],
         [0.3961, 0.2290],
         [0.0677, 0.1873]],

        [[0.5449, 0.3570],
         [0.7489, 0.0524],
         [0.0794, 0.0586]],

        [[0.1462, 0.7153],
         [0.5541, 0.2122],
         [0.2986, 0.1806]],

    

## Training an AutoDecoder_Ensemble with an ensemble of randomly initialized CompNets

The individual ensemble nets are the same CompNets but are initialized with differing initializations to the classifiers independent as possible

In [269]:
ensemble_HP = HyperParameter(lr=0.001, batch_size=16)
ensemble_DS = DirectorySetting(CLASSIFIER_TRAINED_WEIGHT_DIR='./ensemble_classifier_trained_weights')

In [276]:
ensemble1 = train_compnet(ensemble_HP, ensemble_DS, 
                          train_ds=train_encoding_ds, 
                          test_ds=test_encoding_ds,
                          compnet=EnsembleCompNet())


Epoch: 0. batch_idx: 100
Loss:  0.8218167006969452 0.5923093342781067
Epoch: 0. batch_idx: 200
Loss:  0.6958015757799149 0.6790514653921127
Epoch: 0. batch_idx: 300
Loss:  0.5258185368776321 0.7428328293561935
Epoch: 0. batch_idx: 400
Loss:  0.37643086373806 0.6744326055049896
Epoch: 0. batch_idx: 500
Loss:  0.2867506204545498 0.6066402149200439
Epoch: 0. batch_idx: 600
Loss:  0.2583423684537411 0.5624333655834198
------------------ Evaluation Report ------------------
Total Accuracy: 0.9365775827682489
After 157 batches and 2507 test points

Metrics for the same class:
Avg loss: 0.2358952956214832
Precision: 0.8990156762668611
Recall: 0.9836457917830076
F1 Score: 0.9394285714285713

Metrics for the diff class:
Avg loss: 0.48819872926754554
Precision: 0.9819462791721708
Recall: 0.8895093737534903
F1 Score: 0.9334449560485558
Eval:  (37.03556141257286, 76.64720049500465, 2466.0, 2230.0, 41.0, 277.0, 157, 2507)
Epoch: 1. batch_idx: 100
Loss:  0.20541204780340194 0.4753797325491905
Epoch:

## Evaluate the Ensemble CompNets

In [278]:
eval_compnet(ensemble1, test_encoding_ds)

------------------ Evaluation Report ------------------
Total Accuracy: 0.9754686876745113
After 157 batches and 2507 test points

Metrics for the same class:
Avg loss: 0.09116257365887902
Precision: 0.9707740916271722
Recall: 0.9804547267650578
F1 Score: 0.975590394919627

Metrics for the diff class:
Avg loss: 0.1923845513326347
Precision: 0.9802578565672845
Recall: 0.9704826485839649
F1 Score: 0.9753457606734817


(14.312524064444005, 30.204374559223652, 2458.0, 2433.0, 49.0, 74.0, 157, 2507)

## Ensemble CompNets with NNs with increasing depths

In [308]:
ensemble_HP = HyperParameter(lr=0.001, batch_size=16)
ensemble_DS = DirectorySetting(CLASSIFIER_TRAINED_WEIGHT_DIR='./ensemble_classifier_trained_weights')

In [314]:
ensemble_compnet_diff = [
    nn.Sequential(
        nn.Linear(ensemble_HP.encoding_size, 64),
        nn.LeakyReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()),
    nn.Sequential(
        nn.Linear(ensemble_HP.encoding_size, 128),
        nn.LeakyReLU(),
        nn.Linear(128, 64),
        nn.LeakyReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()),
    nn.Sequential(
        nn.Linear(ensemble_HP.encoding_size, 256),
        nn.LeakyReLU(),
        nn.Linear(256, 128),
        nn.LeakyReLU(),
        nn.Linear(128, 64),
        nn.LeakyReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()),
    nn.Sequential(
        nn.Linear(ensemble_HP.encoding_size, 512),
        nn.LeakyReLU(),
        nn.Linear(512, 256),
        nn.LeakyReLU(),
        nn.Linear(256, 128),
        nn.LeakyReLU(),
        nn.Linear(128, 64),
        nn.LeakyReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()),
]

In [317]:
ensemble2 = train_compnet(ensemble_HP, ensemble_DS, 
                          train_ds=train_encoding_ds, 
                          test_ds=test_encoding_ds,
                          compnet=EnsembleCompNet(comp_net=ensemble_compnet_diff,
                                                 num_ensemble=len(ensemble_compnet_diff))
                         )

Epoch: 0. batch_idx: 100
Loss:  0.6069590747356415 0.5226758068799973
Epoch: 0. batch_idx: 200
Loss:  0.5611259976029396 0.39650227546691896
Epoch: 0. batch_idx: 300
Loss:  0.524142044186592 0.3353692851960659
Epoch: 0. batch_idx: 400
Loss:  0.49901885628700254 0.313214952647686
Epoch: 0. batch_idx: 500
Loss:  0.4597411745786667 0.28801489874720576
Epoch: 0. batch_idx: 600
Loss:  0.43802073508501055 0.2499128845334053
------------------ Evaluation Report ------------------
Total Accuracy: 0.9415636218587954
After 157 batches and 2507 test points

Metrics for the same class:
Avg loss: 0.4268379583480252
Precision: 0.9738869863013698
Recall: 0.9074591144794575
F1 Score: 0.9395003097253768

Metrics for the diff class:
Avg loss: 0.23827302759620034
Precision: 0.9133681852128454
Recall: 0.9756681292381332
F1 Score: 0.9434908389585341
Eval:  (67.01355946063995, 37.408865332603455, 2275.0, 2446.0, 232.0, 61.0, 157, 2507)
Epoch: 1. batch_idx: 100
Loss:  0.4035860705375671 0.21816798292100428
E

## Evaluate the Ensemble 2 CompNets

In [318]:
eval_compnet(ensemble2, test_encoding_ds)

------------------ Evaluation Report ------------------
Total Accuracy: 0.9615077782209812
After 157 batches and 2507 test points

Metrics for the same class:
Avg loss: 0.21810709794235836
Precision: 0.9792874896437448
Recall: 0.9429597128041484
F1 Score: 0.9607803292013818

Metrics for the diff class:
Avg loss: 0.10121579959419123
Precision: 0.945
Recall: 0.9800558436378142
F1 Score: 0.9622087331114156


(34.242814376950264,
 15.890880536288023,
 2364.0,
 2457.0,
 143.0,
 50.0,
 157,
 2507)